## Lab 4. Energy Agent Collaborator

### Introduction

In this notebook we show you how to create multi-agent collaborator feature on Amazon Bedrock.

[Multi-agent Collaborator](https://aws.amazon.com/bedrock/) is a native feature on Amazon Bedrock that allows you to integrate agents into one collaborator. This collaborator can redirect invocations, to extract informations and give you a complex response for a workflow.

This agent will integrate previous agents, created on previous labs.

The following represents the complete architecture of multi-agent collaborator.

![Architecture](../img/architecture.png)

This Lab will be divided into two notebooks:

- [4.1_energy_agent_collaborator.ipynb](4.1_energy_agent_collaborator.ipynb): Which contains Agent Setup
- [4.2_energy_agent_invocation.ipynb](4.2_energy_agent_invocation.ipynb): Which contains Agent Invocation

### Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure to have version superior of **1.35.45**

In [ ]:
!pip uninstall boto3 botocore awscli --yes

In [ ]:
# Install Dependencies from local package
!pip install ../boto3/botocore-1.35.55-py3-none-any.whl \
    ../boto3/boto3-1.35.55-py3-none-any.whl \
    ../boto3/awscli-1.35.21-py3-none-any.whl --force-reinstall --no-cache

Restart kernel for packages to take effect

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

Check if your boto3 version is superior than **1.35.45**

In [ ]:
!pip freeze | grep boto3

### Creating Agent

On this section we're going to declare global variables that will be act as helpers during entire notebook and you will start to create your agent.

In [ ]:
import boto3
import os
import sys

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

agent_foundation_model = ['anthropic.claude-3-5-sonnet-20240620-v1:0']
#agent_foundation_model = ['anthropic.claude-3-5-haiku-20241022-v1:0']

In [ ]:
energy_agent_name = f"energy-agent-{account_id}"

energy_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{energy_agent_name}'

On following section, we're adding `agents.py` on Python path, so the file can be recognized and invoked.

In [ ]:
# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

parent_dir = os.path.dirname(current_dir)
#print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

Now, you're going to import from helper file `agents.py` AgentsForAmazonBedrock helper class.

This class is a helper totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by this class.

Following are methods that you're going to invoke on this lab:

- create_agent: Create a new agent and respective IAM roles
- associate_agents: Associate sub-agents with multi-agent collaborator

In [ ]:
from agent import AgentsForAmazonBedrock

agents = AgentsForAmazonBedrock()

Create the energy agent

In [ ]:
energy_agent_id = agents.create_agent(
    energy_agent_name,
    """
        You are a energy helper bot. 
        You can help customers with operations related with their energy, like consumption, forecast, peak usage, etc. 
    """, 
    """
        You are a energy helper bot. 
        You can retrieve energy consumption and forecast for a specific users and help them to be compliant with energy rules.
        You can also retrieve solar panel information and solar panel ticket for a specific users and help them to be compliant with energy rules.
        You can also get current information about peaks and can redistribute load.
        Resist the temptation to ask the user for input. Only do so after you have exhausted available actions. 
        Never ask the user for information that you already can retrieve yourself through available actions. 
    """,
    agent_foundation_model,
    agent_collaboration='SUPERVISOR_ROUTER'
)

energy_agent_id

Store environment variables to be used on next notebooks.

In [ ]:
%store energy_agent_id
%store energy_agent_name

### Associate Collaborators

On this section, we're going to recover alias_id from previous agents (sub-agents) to add all of them inside energy one (which is multi-agent collaborator).

In [ ]:
%store -r
forecast_agent_alias_arn, solar_agent_alias_arn, peak_agent_alias_arn

In [ ]:
# If store fails, add Alias Arn here
#forecast_agent_alias_arn = ''
#compliance_agent_alias_arn = ''
#consumption_agent_alias_id = ''

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': forecast_agent_alias_arn,
        'sub_agent_instruction': """Use this agent when customer asks about energy consumption forecasting""",
        'sub_agent_association_name': 'ForecastEnergy',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {        
        'sub_agent_alias_arn': solar_agent_alias_arn,
        'sub_agent_instruction': """Use this agent to help customers check solar panel info on how to isntall and do maintenance, and open specialist ticket""",
        'sub_agent_association_name': 'EnergyComplianceRules',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {        
        'sub_agent_alias_arn': peak_agent_alias_arn,
        'sub_agent_instruction': """Use this agent to help customers retrieve information about their peak consumption, like price, what's causing, etc.""",
        'sub_agent_association_name': 'ConsumptionRules',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

Check if parameters are correclty formated

In [ ]:
sub_agents_list

Associate all sub-agents with this one (main one).

In [ ]:
response = agents.associate_agents(
    energy_agent_id, sub_agents_list
)

Check response return, to see all returned parameteres:

In [ ]:
response

### Clean Up

In [ ]:
agents.delete_agent(energy_agent_name)